# Inheriting properties from built-in classes
---

### Introduction

Python gives you the ability to __create a class that inherits properties from any Python built-in class__ in order to get a new class that can enrich the parent's attributes or methods.

As a result, your newly-created class has the advantage of all of the well-known functionalities inherited from its parent or even parents and you can still access those attributes and methods.

---


### Creating a custom list class

In the following example, we’ll create an implementation of our own list class, which will only accept elements of the integer type. But, wait – why might you need such an object?

![Alt text](<inheritance build-in classes.png>)

The example does not bring you anything that has not been seen so far since it performs the following:
1. You create a class that inherits from another class.
2. Methods of the inherited class are replaced.
3. The methods perform more specialized validations on the arguments.
4. When the validation is finished, the original method of the parent class is called.

With the three overwritten methods of the list class (extend, __setitem__, append) all input paths for new values with additional validation are already controlled.

---

### Well stop talking and start coding!

In [1]:
class IntegerList(list):

    @staticmethod
    def check_value_type(value):
        if type(value) is not int:
            raise ValueError('Not an integer type')

    def __setitem__(self, index, value):
        IntegerList.check_value_type(value)
        list.__setitem__(self, index, value)

    def append(self, value):
        IntegerList.check_value_type(value)
        list.append(self, value)

    def extend(self, iterable):
        for element in iterable:
            IntegerList.check_value_type(element)

        list.extend(self, iterable)


int_list = IntegerList()

int_list.append(66)
int_list.append(22)
print('Appending int elements succeed:', int_list)

int_list[0] = 49
print('Inserting int element succeed:', int_list)

int_list.extend([2, 3])
print('Extending with int elements succeed:', int_list)

try:
    int_list.append('8-10')
except ValueError:
    print('Appending string failed')

try:
    int_list[0] = '10/11'
except ValueError:
    print('Inserting string failed')

try:
    int_list.extend([997, '10/11'])
except ValueError:
    print('Extending with ineligible element failed')

print('Final result:', int_list)

Appending int elements succeed: [66, 22]
Inserting int element succeed: [49, 22]
Extending with int elements succeed: [49, 22, 2, 3]
Appending string failed
Inserting string failed
Extending with ineligible element failed
Final result: [49, 22, 2, 3]


__NOTE:__ The `__setitem__` method is used by classes that work with indexed objects (lists, dictionaries, tuples). It is invoked (and receives the index and the value) when calling the object through [].

__NOTE 2:__`__getitem__` does the opposite – it is invoked when the object is called through [] and receives the index as an argument.

---

### An example with dictionaries

Lets imagine that we want to create an special class that

The code will be the following:

In [4]:
import random


class IBANValidationError(Exception):
    pass


class IBANDict(dict):
    def __setitem__(self, _key, _val):
        if IBANDict.validateIBAN(_key):
            super().__setitem__(_key, _val)

    def update(self, *args, **kwargs):
        for _key, _val in dict(*args, **kwargs).items():
            self.__setitem__(_key, _val)

    @staticmethod
    def validateIBAN(iban: str):
        iban = iban.replace(' ', '')

        if not iban.isalnum():
            raise IBANValidationError("You have entered invalid characters.")

        elif len(iban) < 15:
            raise IBANValidationError("IBAN entered is too short.")

        elif len(iban) > 31:
            raise IBANValidationError("IBAN entered is too long.")

        else:
            iban = (iban[4:] + iban[0:4]).upper()
            iban2 = ''
            for ch in iban:
                if ch.isdigit():
                    iban2 += ch
                else:
                    iban2 += str(10 + ord(ch) - ord('A'))
            ibann = int(iban2)

            if ibann % 97 != 1:
                raise IBANValidationError("IBAN entered is invalid.")

            return True


my_dict = IBANDict()
keys = ['GB72 HBZU 7006 7212 1253 00', 'FR76 30003 03620 00020216907 50', 'DE02100100100152517108']

for key in keys:
    my_dict[key] = random.randint(0, 1000)

print('The my_dict dictionary contains:')
for key, value in my_dict.items():
    print("\t{} -> {}".format(key, value))

try:
    my_dict.update({'dummy_account': 100})
except IBANValidationError:
    print('IBANDict has protected your dictionary against incorrect data insertion')


The my_dict dictionary contains:
	GB72 HBZU 7006 7212 1253 00 -> 900
	FR76 30003 03620 00020216907 50 -> 260
	DE02100100100152517108 -> 317
IBANDict has protected your dictionary against incorrect data insertion


---